데이터 불러오기 + Resize 하기
-------------

In [1]:
from PIL import Image
import os, glob

# resize
def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")


image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

5420  images to be resized.
5420  images resized.
가위 이미지 resize 완료!
5302  images to be resized.
5302  images resized.
바위 이미지 resize 완료!
5370  images to be resized.
5370  images resized.
보 이미지 resize 완료!


데이터 라벨링 
-------------


In [2]:
import numpy as np

def load_data(img_path, number):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=number
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path, 16092)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))


학습데이터(x_train)의 이미지 개수는 16092 입니다.
x_train shape: (16092, 28, 28, 3)
y_train shape: (16092,)


딥러닝 네트워크 설계하기
-------------

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_channel_1=16
n_channel_2=32
n_dense=32
n_train_epoch=20

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))
          
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9

딥러닝 네트워크 학습시키기
-------------

In [4]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs = n_train_epoch)

Epoch 1/20
503/503 [==============================] - 9s 9ms/step - loss: 1.0256 - accuracy: 0.4526
Epoch 2/20
503/503 [==============================] - 1s 3ms/step - loss: 0.6330 - accuracy: 0.7323
Epoch 3/20
503/503 [==============================] - 1s 3ms/step - loss: 0.4325 - accuracy: 0.8362
Epoch 4/20
503/503 [==============================] - 1s 3ms/step - loss: 0.3052 - accuracy: 0.8833
Epoch 5/20
503/503 [==============================] - 1s 3ms/step - loss: 0.2268 - accuracy: 0.9161
Epoch 6/20
503/503 [==============================] - 1s 3ms/step - loss: 0.1704 - accuracy: 0.9392
Epoch 7/20
503/503 [==============================] - 1s 3ms/step - loss: 0.1399 - accuracy: 0.9510
Epoch 8/20
503/503 [==============================] - 1s 3ms/step - loss: 0.1057 - accuracy: 0.9658
Epoch 9/20
503/503 [==============================] - 1s 3ms/step - loss: 0.0794 - accuracy: 0.9737
Epoch 10/20
503/503 [==============================] - 1s 3ms/step - loss: 0.0626 - accuracy: 0.9810

테스트 이미지 Resize 하기
-------------

In [5]:
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
resize_images(image_dir_path)

print("가위 테스트 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
resize_images(image_dir_path)

print("바위 테스트 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path)

print("보 테스트 이미지 resize 완료!")

100  images to be resized.
100  images resized.
가위 테스트 이미지 resize 완료!
100  images to be resized.
100  images resized.
바위 테스트 이미지 resize 완료!
100  images to be resized.
100  images resized.
보 테스트 이미지 resize 완료!


test 데이터 준비
-------------

In [6]:
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path, 300)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


test_accuracy 측정
----------

In [7]:
test_loss, test_accuracy = model.evaluate(x_test_norm,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 2s - loss: 0.0584 - accuracy: 0.9767
test_loss: 0.05839907377958298 
test_accuracy: 0.9766666889190674


회고
------

1. 이미지 분류기 모델이 성공적으로 만들어졌는가?

keras의 모델을 이용하여 하이퍼 마라미터 값을 변경하며 학습시켰습니다.

처음 시도하였을때 정확히 이해하지 못하고 lms를 참고해 대입하면서 제작하였고

TF에서 학습한 내용을 토대로 과정을 이해하려고 노력했습니다

2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가?

Min-Max Normalization을 사용하여 0~1사이로 정규화 하였으며

트레인 데이터셋을 추가하여 오버피팅을 극복하려고 노력하였습니다.

3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?

데이터셋을 가위,바위,보 최종 도합 16092개를 사용하였습니다

처음 데이터셋은 300개로 시작하였으나 test accuracy 값이 0.3467로 나와

하이퍼 파라미터 수정->train 데이터 추가->반복

하는 과정으로 시도하였습니다.

중간에 도합 5400개로 시도하였을때 테스트셋을 추가해봤는데 

어떠한 사진이라도 accuracy가 잘 나와야한다는생각에 테스트셋을 많이 늘려보기도 하고

랜덤으로 갯수를 추출하여 테스트셋을 넣어보기도 하였습니다

최종적으로는 기준 이상을 넘긴 0.9767 수치가 나왔습니다.
